In [1]:
# imports
import atc_dataloader, atc_model
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def train_model(model, dataset_train, dataset_test, optimizer, epochs=10):
    
    loss = torch.nn.MSELoss()
    
    for epoch in range(epochs):
        # set the model to train mode
        model.train()
        for batch in dataset_train:
            # get the data from the batch
            data = batch['data'].to(device)
            target = batch['target'].to(device)
            
            # zero the gradients
            optimizer.zero_grad()
            
            # forward pass

In [4]:
# load data
TRAIN_IN="data/train_in.csv"
TRAIN_OUT="data/train_out.csv"

data_train = atc_dataloader.ATCDataset(TRAIN_IN, TRAIN_OUT)

TEST_IN="data/test_in.csv"
TEST_OUT="data/test_out.csv"

data_test = atc_dataloader.ATCDataset(TEST_IN, TEST_OUT)

In [20]:
model = atc_model.BaseNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

train_model(model, dataset_train=data_train, dataset_test=data_test, optimizer=optimizer, epochs=100)

11862489
